<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Optional: login to GenePattern
</div>

In [1]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

## RNASeq workflow for Rady Children's neuro-oncology tumor board

This notebook contains all the routine analyses the Mesirov lab has been doing for the Rady's pediatric neuro-oncology tumor board. Its purpose is to enable another a collaborator to do these analyses quickly and reproducibly.

## User input

Select parameters before running the rest of the notebook.

<div class="alert alert-danger">
EDWIN: add option to change input and control!
</div>

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Select parameters before running the rest of the notebook.
</div>

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import readline # required for rpy2 extension
%load_ext rpy2.ipython
import os
import sys
import subprocess
import warnings
import re
sys.path.append('/build/')

import pandas as pd
import numpy as np
import rpy2
from sklearn.ensemble import RandomForestClassifier
from tumor_classification.medulloblastoma import classify_cavalli, classify_cho, classify_northcott
from slides import make_medullo_classification_slide, make_discover_workflow_slide, make_exp_drug_ranking_results_slide, make_intersection_slide
from utils import source_and_update_env_vars

import pickle

def rmagic_warning(
    message,
    category = rpy2.rinterface.RRuntimeWarning,
    filename = '',
    lineno = -1,
    file=None, 
    line=None):
    print(message)
default_showwarning = warnings.showwarning

class Bunch(object):
  def __init__(self, adict):
    self.__dict__.update(adict)

@genepattern.build_ui(parameters={
    "output_var": {
        "default": "setup",
        "hide": False,
    },
    "case_id": {"type": "text",
                "description": "The name of the case, e.g., 'case17'",},
    "patient_dir": {"type": "text",
                    "description": "The name of the patient directory, e.g. '18-10716_tumor-normal' (quotes are required)",},
    "is_medullo": {"type": "bool",
                   "description": "Whether or not this sample has been classified as medulloblastoma",},
    "control": {"type": "choice",
                "description": "Whether or not to use a custom control",
                "choices": {
                    "original": "original",
                    "custom": "custom",
                            }
               },
    "custom_control_expression": {"type": "file",
                           "kinds": ["gct"],
                           "description": "The file (or path to the GCT file) which contains the gene expression of the custom control.",
                           "default":None},
})
def read_user_input(case_id, patient_dir, is_medullo=False, control='original',custom_control_expression=None):
    # Select control for DiSCoVER and Connectivity Map
    # Generally, if the tumor is a medulloblastoma, we use `cerebellar_stem` (comment the `neural_stem` line).
    # And if it is any other kind of brain tumor, we use `neural_stem`.
    
    if control == 'original':
        expression_control = 'cerebellar_stem' if is_medullo else 'neural_stem'
    elif control == 'custom':
        expression_control = 'custom_control'
    else:
        print('Unexpected value for variable named control, value:', control)
        
    if (len(custom_control_expression) is not 0) and (control is not 'custom'):
        print("Reminder: if you want to use a custom control expresion, you must set control to 'custom'")

    base_dir = os.getcwd()
    utilities_dir = '/build'
    patients_dir = os.path.join(base_dir, 'patients')
    in_dir = os.path.join(patients_dir, patient_dir)
    out_dir = in_dir
    os.makedirs(out_dir, exist_ok=True)

    platform = sys.platform
    if platform.startswith('linux'):
        os_string = 'linux'
    elif platform == 'darwin':
        os_string = 'mac'
    else:
        raise ValueError('Platform "{}" not supported'.format(platform))

    # RNASeq quantification
    kallisto_dir = '/build/kallisto'
    kallisto_path = os.path.join(kallisto_dir, 'kallisto_{}-v0.44.0/kallisto'.format(os_string))
    transcriptome_index_path = os.path.join(kallisto_dir, 'GRCh38.ensembl.transcriptome.idx')
    local_fastqs_dir = os.path.join(in_dir, 'fastqs')
    os.makedirs(local_fastqs_dir, exist_ok=True)
    patient_gexp_file = os.path.join(out_dir, 'gene_abundance.sleuth.csv')

    # Medulloblastoma classification
#     from sklearn.ensemble import RandomForestClassifier
#     from tumor_classification.medulloblastoma import classify_cavalli, classify_cho, classify_northcott
    medullo_classify_out_dir = os.path.join(out_dir, 'medulloblastoma_classification')
    if not os.path.exists(medullo_classify_out_dir):
        os.mkdir(medullo_classify_out_dir)
    cavalli_subgroup_file = os.path.join(medullo_classify_out_dir, 'cavalli_subgroups.csv')
    cavalli_subgroup_direct_file = os.path.join(medullo_classify_out_dir, 'cavalli_subgroups_direct.csv')
    cavalli_subtype_file = os.path.join(medullo_classify_out_dir, 'cavalli_subtypes.csv')
    cho_subtype_file = os.path.join(medullo_classify_out_dir, 'cho_subtypes.csv')
    cho_subgroup_file = os.path.join(medullo_classify_out_dir, 'cho_subgroups.csv')
    northcott_subgroup_file = os.path.join(medullo_classify_out_dir, 'northcott_subgroups.csv')

    drug_suggestion_out_dir = os.path.join(out_dir, 'drug_suggestions')
    os.makedirs(drug_suggestion_out_dir, exist_ok=True)

    # DiSCoVER
    discover_out_dir = os.path.join(drug_suggestion_out_dir, 'discover/{}'.format(expression_control))
    os.makedirs(discover_out_dir, exist_ok=True)
    discover_heatmap_file = os.path.join(discover_out_dir, 'ctrp.png')
    full_discover_results_file = os.path.join(discover_out_dir, 'discover.all.csv')
    rdrugs_discover_file = os.path.join(discover_out_dir, '{}.discover.{}.reasonable.annotated.csv'.format(case_id, expression_control))

    # Connectivity Map
    cmap_out_dir = os.path.join(drug_suggestion_out_dir, 'cmap/{}'.format(expression_control))
    os.makedirs(cmap_out_dir, exist_ok=True)
    cmap_all_ranked_drugs_file = os.path.join(cmap_out_dir, '{}.cmap.{}.all.csv'.format(case_id, expression_control))
    cmap_reasonable_ranked_drugs_file = os.path.join(cmap_out_dir, '{}.cmap.{}.reasonable.annotated.csv'.format(case_id, expression_control))
    
    # Powerpoint for MTB
#     from slides import make_medullo_classification_slide, make_discover_workflow_slide, make_exp_drug_ranking_results_slide, make_intersection_slide
    mtb_ppt_file = os.path.join(out_dir, '{}.mtb_slides.pptx'.format(case_id))

    # DNANexus
    dx_source_path = os.path.join(utilities_dir, 'dx-toolkit/environment')
    dnanexus_project = 'UW_UCSD_RNAseq_collaboration_share'
    # Replace the contents of this file with your own DNANexus token.
    dnanexus_token_file = os.path.join(base_dir, 'dnanexus_token.txt')
    # To use the dx command, we must update some environment variables. 
    # From the command line, this is done with source dx-toolkit/environment, 
    # but from Python we have to use a workaround, because normally any changes 
    # to environment variables done in a subprocess are not reflected in the 
    # parent process. The workaround runs the source command in a subprocess, 
    # fetches the environment variables from the subprocess and updates those 
    # of the parent process.
#     from utils import source_and_update_env_vars
    source_and_update_env_vars(dx_source_path)
    
    out = {"case_id": case_id,
                 "patient_dir": patient_dir,
                 "is_medullo": is_medullo}
    out['expression_control'] = expression_control
    out['custom_control_expression'] = custom_control_expression
    out['dnanexus_token_file'] = dnanexus_token_file
    out['local_fastqs_dir'] = local_fastqs_dir
    out['dnanexus_project'] = dnanexus_project
    out['local_fastqs_dir'] = local_fastqs_dir
    out['transcriptome_index_path'] = transcriptome_index_path
    out['kallisto_path'] = kallisto_path
    out['kallisto_dir'] = kallisto_dir
    out['out_dir'] = out_dir
    out['r_out_dir'] = out_dir.replace('\\',r'\\')
    out['patient_gexp_file'] = patient_gexp_file
    out['in_dir'] = in_dir
    out['cavalli_subgroup_file'] = cavalli_subgroup_file
    out['cavalli_subtype_file'] = cavalli_subtype_file
    out['cavalli_subgroup_direct_file'] = cavalli_subgroup_direct_file
    out['cho_subgroup_file'] = cho_subgroup_file
    out['cho_subtype_file'] = cho_subtype_file
    out['northcott_subgroup_file'] = northcott_subgroup_file
    out['mtb_ppt_file'] = mtb_ppt_file
    out['expression_control'] = expression_control
    out['full_discover_results_file'] = full_discover_results_file
    out['discover_out_dir'] = discover_out_dir
    out['discover_heatmap_file'] = discover_heatmap_file
    out['rdrugs_discover_file'] = rdrugs_discover_file
    out['cmap_out_dir'] = cmap_out_dir
    out['cmap_all_ranked_drugs_file'] = cmap_all_ranked_drugs_file
    out['cmap_reasonable_ranked_drugs_file'] = cmap_reasonable_ranked_drugs_file
    out['mtb_ppt_file'] = mtb_ppt_file    
    print('Setup done!')
    
    return Bunch(out)

UIBuilder(function_import='read_user_input', name='read_user_input', params=[{'name': 'case_id', 'label': 'cas…

<div class="well">
Running all cells below this point will execute all the analyses except for one: the Connectivity Map analysis at the end of the notebook, which requires two manual steps.
</div>

# Download RNAseq data

In [3]:
def log(message,extra='==>'):
    print(extra,message)
    return


def preprocess_rna_seq(setup):
    if not os.path.exists(setup.transcriptome_index_path):
        kallisto_idx_command = '{} index -i {} {}/Homo_sapiens.GRCh38.*'.format(kallisto_path, transcriptome_index_path, kallisto_dir)
        subprocess.check_call(kallisto_idx_command, shell=True).decode('utf-8')
    
    fq_subdirs = [os.path.join(setup.local_fastqs_dir, subdir) for subdir in os.listdir(setup.local_fastqs_dir)]
    ordered_fastqs = []
    for fq_subdir in fq_subdirs:
        if '.DS_Store' not in fq_subdir:
            fq_files = sorted([os.path.join(fq_subdir, file) for file in os.listdir(fq_subdir) if file.endswith('fastq.gz')])
            ordered_fastqs.extend(fq_files)
    fastqs_str = '" "'.join(ordered_fastqs)
    fastqs_str = '"'+fastqs_str+'"'
    
    command = '"{}" quant -i "{}" -o "{}" --bias -b 2 {}'.format(setup.kallisto_path, setup.transcriptome_index_path, setup.out_dir, fastqs_str)
    #print(command) # if you want to execute it outside this notebook in a console
    
    subprocess.check_call(command, shell=True)#.decode('utf-8')    
    return


def run_R(command, rcript='temp_rscript.R', rlog='temp.log'):
    print("Running an R command, this may take a while and will only print output at the end of the run.")
    with open(rcript, 'w') as f:
        f.write(command)

    subprocess.call(f'Rscript {rcript} > {rlog} 2>&1 || cat {rlog}', shell=True)

    with open(rlog, 'r') as f:
        for line in f.readlines():
            print(line)
    return

def run_sleuth(setup):
    warnings.showwarning = rmagic_warning # to only print the warning text, not text + returned warning object
    from rpy2.robjects import numpy2ri
    numpy2ri.activate()
    r_command = f"""
        pkg.is.installed <- function(mypkg)
        {{
            return(mypkg %in% rownames(installed.packages()))
        }}
        source("http://bioconductor.org/biocLite.R")
        if(!pkg.is.installed('biomaRt'))
        {{
            biocLite('biomaRt')
        }}
        library("biomaRt")
        mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
          dataset = "hsapiens_gene_ensembl",
          host = 'www.ensembl.org')
        t2g <- biomaRt::getBM(attributes = c("ensembl_transcript_id", "external_gene_name"), mart = mart)
        t2g <- dplyr::rename(t2g, target_id = ensembl_transcript_id, ext_gene = external_gene_name)

        if(!pkg.is.installed('rhdf5'))
        {{
            biocLite('rhdf5')
        }}
        if(!pkg.is.installed('devtools'))
        {{
            install.packages('devtools')
        }}
        if(!pkg.is.installed('sleuth'))
        {{
            devtools::install_github("pachterlab/sleuth")
        }}
        library(ggplot2) # req'd by sleuth
        library(dplyr, warn.conflicts = FALSE) # req'd by sleuth; masks some functions, but this doesn't affect us
        library("sleuth")
        s2c = do.call(rbind.data.frame, list(c('{setup.case_id}', '{setup.case_id}', '{setup.out_dir}')))
        colnames(s2c) <- c('sample', 'condition', 'path')
        s2c[] <- lapply(s2c, as.character)
        so <- sleuth_prep(s2c, ~condition, target_mapping=t2g, aggregation_column='ext_gene')
        write.csv(so$obs_norm[, c(2,4)], '{setup.patient_gexp_file}', quote=FALSE, row.names=FALSE)

        """
    run_R(r_command)
    numpy2ri.deactivate()
    # Return to default warning handling.
    warnings.showwarning = default_showwarning
    return


@genepattern.build_ui(parameters={
    "setup": {"default": "setup",
              "hide": False,
              "description": "The variable which has the setup information"},
    "output_var": {
        "default": "setup",
        "hide": True,
    },
})
def download_and_preprocess_rnaseq(setup):
    log('About to download fastqfiles from DNA Nexus. This may take a while.')
    with open(setup.dnanexus_token_file, 'r') as f:
        dnanexus_token = f.readline().strip()
    login_command = 'dx login --token {} --noprojects; dx select {}'.format(dnanexus_token, setup.dnanexus_project)
    # subprocess.check_output('ls', shell=True).decode('utf-8').strip()
    subprocess.check_output(login_command, shell=True).decode('utf-8').strip()

    find_fastq_command = 'dx find data --name "*.fastq.gz" --path {}:{}'.format(setup.dnanexus_project, setup.patient_dir)
    find_fastq_return_lines = subprocess.check_output(find_fastq_command, shell=True).decode().strip().split('\n')
    re_string = '.*(/{}/.*\.fastq.gz) .*'.format(setup.patient_dir)
    fastq_path_re = re.compile(re_string)
    remote_fastq_paths = []
    local_fastq_subdirs = []

    for line in find_fastq_return_lines:
        search = fastq_path_re.search(line)
        remote_fastq_path = search.group(1)
        remote_fastq_paths.append(remote_fastq_path)
        fastq_subdir_path = os.path.dirname(remote_fastq_path)
        fastq_subdir = os.path.basename(fastq_subdir_path)
        local_fastq_subdir = os.path.join(setup.local_fastqs_dir, fastq_subdir)
        os.makedirs(local_fastq_subdir, exist_ok=True)
        local_fastq_subdirs.append(local_fastq_subdir)

    for remote_fastq_path, local_fastq_subdir in zip(remote_fastq_paths, local_fastq_subdirs):
        download_command = 'dx download "{}" -o "{}"'.format(remote_fastq_path, local_fastq_subdir)
        print('\t'+download_command)
        try:
            a=subprocess.check_output(download_command, shell=True)
        except subprocess.CalledProcessError as e:
            print('\tEncountered a dx error, this likely means you already have the file indicated above.')
            print('\tContinuing...\n')
            continue
    log('Done downloading the fastq files.')
    log('Preprocessing RNASeq data now:')
    log('Using kallisto to compute transcript abundance.')
    preprocess_rna_seq(setup)
    log('Done with tanscript abundance.')
    log('Using sleuth to aggregate transcript abundance into gene abbundance.')
    run_sleuth(setup)
    patient_exp = pd.read_csv(setup.patient_gexp_file, index_col=0).T
    patient_exp.index = [setup.case_id]
    setup.patient_exp = patient_exp
    patient_exp.to_csv(setup.patient_gexp_file)
    log('Habemus Genus Expressium *release the white smoke*')
    
    log('Done preprocessing!')
    return setup

UIBuilder(function_import='download_and_preprocess_rnaseq', name='download_and_preprocess_rnaseq', params=[{'n…

# Classify the tumor by medulloblastoma subgroup and subtype

In [10]:
print("what")

what


In [4]:
@genepattern.build_ui(
    description="This function classifies a medulloblastoma sample into a subgroup. Non-medulloblastoma samples are ignored.",
    parameters={
    "setup": {"default": "setup",
              "hide": False,
              "description": "The variable which has the setup information"},
    "output_var": {
        "default": "setup",
        "hide": True,
    },
})
def classify_sample(setup):
    # We have three datasets we can use to classify based on expression:
    # - [Cavalli et al. 2017](http://www.sciencedirect.com/science/article/pii/S1535610817302015) cohort. This cohort includes 763 tumors, and was used to define 12 finer-grained subtypes nested in the 4 subgroups. Both expression and methylation data are available.

    # - [Cho et al. 2011](http://www.mesirovlab.org/medulloblastoma/cho/) cohort. This paper identified two subtypes within G3 and two within G4, for a total of 6. It contains 194 tumors.

    # - [Northcott et al. 2017](http://www.nature.com/nature/journal/v547/n7663/full/nature22973.html) expression data (shared by Sebastian). The labels we have for this data are of the 4 basic subgroups only. There are 223 tumors.

    # When finer-grained subtypes are known, we perform the finer-grained classification first and also collapse the subtypes to the 4 basic subgroups, so as to report both subtype and subgroup probabilities. Classification is done using random forests.

    # Since the patient data are from the same platform and contain the same features each time, we can use pre-fit models to classify them. The classification methods also have a fallback in case the data looks different.

    # The tumor board is arranging for methylation data to be obtained from patient samples as well, since it seems it may be more informative than expression. Methylation data would also allow comparison to a large and variety collection of brain tumors, currently available through a DKFZ [web portal](https://www.molecularneuropathology.org/mnp).

    if setup.is_medullo:
        # Read in patient's gene-level RNASeq TPM data
        patient_exp = pd.read_csv(setup.patient_gexp_file, index_col=0)

        cavalli_subgroups, cavalli_subtypes = classify_cavalli(patient_exp)
        cavalli_subgroups.to_csv(setup.cavalli_subgroup_file)
        cavalli_subtypes.to_csv(setup.cavalli_subtype_file)

        cho_subgroups, cho_subtypes = classify_cho(patient_exp)
        cho_subtypes.to_csv(setup.cho_subtype_file)
        cho_subgroups.to_csv(setup.cho_subgroup_file)

        northcott_subgroups = classify_northcott(patient_exp)
        northcott_subgroups.to_csv(setup.northcott_subgroup_file)

        make_medullo_classification_slide(setup.mtb_ppt_file,
                                          setup.cavalli_subgroup_file,
                                          setup.cavalli_subtype_file,
                                          setup.cho_subgroup_file,
                                          setup.cho_subtype_file,
                                          setup.northcott_subgroup_file)
    else:
        print('Not a medulloblastoma sample. Nothing to do.')
            
    return setup

UIBuilder(description='This function classifies a medulloblastoma sample into a subgroup. Non-medulloblastoma …

In [12]:
pickle.dump(setup, file=open(setup.case_id+'_TESTING_NSC194_NOW.p','wb'))

In [13]:
setup = pickle.load(file=open(setup.case_id+'_TESTING_NSC194_NOW.p','rb'))

# Suggest drugs based on RNAseq data (DiSCoVER)

In [14]:
setup.custom_control_expression

'NSC194.gct'

In [15]:
def load_custom_control_exp(filename):
    print(filename)
    df = pd.read_table(filename, header=2, sep='\t', index_col=0)
    return df.drop('Description', axis=1).ix[:,0].to_frame()

In [5]:
def split_discover_dataframe(df, min_score=0):
    # This is super inefficient for larger DataFrames, but let's worry about efficiency later
    
    out = pd.DataFrame(columns=['moa','GDSC','CTRP','CCLE','drug'])
    
    for index, row in df.iterrows():
        dataset = row['drug'].split('_')[0].lower()
        drug = row['drug'].split('_')[1].lower()

        if dataset == 'gdsc':
            out.loc[drug,'GDSC'] = row['score']
        elif dataset == 'ctrp':
            out.loc[drug,'CTRP'] = row['score']
        elif dataset == 'ccle':
            out.loc[drug,'CCLE'] = row['score']
        out.loc[drug,'drug'] = row['drug']
        out.loc[drug,'moa'] = row['moa']
    
    #     df['database'] = [drug.split('_')[0].upper() for drug in df['drug']]
    #     df['drug'] = [drug.split('_')[1].lower() for drug in df['drug']]
    #     df = df[df['score']>0]  # It feels inneficient to do this last, but Pandas complaints if I do it earlier.    
    #     out = df[~df['drug'].duplicated(keep='first')].drop('score', axis=1,inplace=False)
    #     out['Drug'] = np.unique([drug.split('_')[1].lower() for drug in df['drug']])
    return out


sign_to_letter = {
    1:"+",
    -1:"-",
    '1.0':"+",
    '-1.0':"-",
    'nan':'.',
    '0.0':'.',
    '0':'.',
    0:'.',
}

def supporting_evidence(row):
    # assuming only three columns
    return sign_to_letter[str(np.sign(row[0]))]+sign_to_letter[str(np.sign(row[1]))]+sign_to_letter[str(np.sign(row[2]))]


def rank_drugs_discover(df):
    df['score'] = df.drop(['moa','drug'],axis=1,inplace=False).mean(axis=1,skipna=True).round(3)
    df['evidence'] = df.drop(['moa','score'],axis=1).apply(supporting_evidence, axis=1)
    return df.sort_values(by=['score'],ascending=False,axis=0)


@genepattern.build_ui(
  description="Run DiSCoVER on the provided sample and control.",
  parameters={
    "setup": {"default": "setup",
              "hide": False,
              "description": "The variable which has the setup information"},
    "output_var": {
        "default": "setup",
        "hide": True,
    },
})
def run_discover(setup):
    from rpy2.robjects import numpy2ri
    numpy2ri.activate()
    from drug_suggestion.expression.discover import discover_from_expression, plot_discover_from_expression
    from drug_suggestion.expression.controls import load_control_exp
    patient_exp = pd.read_csv(setup.patient_gexp_file, index_col=0)
    if setup.expression_control == 'custom_control':
        log('Loading expression from custom control')
        control_exp = load_custom_control_exp(setup.custom_control_expression)
        print(control_exp)
    else:
        control_exp = load_control_exp(setup.expression_control)
    log("About to perform DiSCoVER.")
    discover_results = discover_from_expression(exp=patient_exp, 
                                                control_exp=control_exp, 
                                                verbose=True)
    log("DiSCoVER done!")
    numpy2ri.deactivate()
    log('Saving results to file.')
    # display(discover_results)
    discover_results.T.sort_values(by=setup.case_id, ascending=False).to_csv(setup.full_discover_results_file)
    log("Saving done!")
    log("Restricting to clinically relevant drugs.")
    #Not all drugs in CCLE, CTRP, and GDSC are realistic candidates for treatment. We compiled a list of medications that are FDA-approved or in late-stage clinical trials, and Dr. Wechsler-Reya curated it to include only those that are relevant for treating brain tumors. Here we limit the results to these drugs and add Dr. Wechsler-Reya's mechanism-of-action annotations. To enable comparison of drug lists, drugs from the different sources have been mapped to PubChem compound IDs (CIDs) using [PubChemPy](http://pubchempy.readthedocs.io/en/latest/).
    from drug_suggestion.drug_annotation import subset_to_reasonable_drugs
    from drug_suggestion.expression.discover import load_discover_drug_to_cids
    disco2cid = load_discover_drug_to_cids()
    reasonable_results = subset_to_reasonable_drugs(discover_results, 
                                                disco2cid, 
                                                out_prefix='discover.{}'.format(setup.expression_control), 
                                                out_dir=setup.discover_out_dir)
    log('Done restricting to clinically relevant drugs!')
    
#     log('making a discover illustrative method')
#     from drug_suggestion.expression.discover import plot_discover_from_expression
#     plot_discover_from_expression(case_id, 
#                                   discover_results, 
#                                   exp=patient_exp,
#                                   control_exp=control_exp,
#                                   cl='ctrp',
#                                   out_file=discover_heatmap_file)
#     make_discover_workflow_slide(mtb_ppt_file, discover_heatmap_file)
    log('Making the DiSCoVER powerpoint.')
    rdrugs_discover = pd.read_csv(setup.rdrugs_discover_file, index_col=None)
    df = split_discover_dataframe(df=rdrugs_discover)
    df = rank_drugs_discover(df)
#     df.head()
    make_exp_drug_ranking_results_slide(setup.mtb_ppt_file, df.head(20), setup.expression_control, method='DiSCoVER')
    log('Done making the DiSCoVER powerpoint slide!')
    log('Savig the variables to a file.')
    setup.discover_results = discover_results
    setup.disco2cid = disco2cid
    setup.control_exp = control_exp
    setup.reasonable_results = reasonable_results
    setup.df = df
    pickle.dump(setup, file=open(setup.case_id+'_DISCoVER.p','wb'))
    log('Done savig the variables to a file!')
    
    log('Done with all the taks in this cell. Move along.')
    return setup

UIBuilder(description='Run DiSCoVER on the provided sample and control.', function_import='run_discover', name…

## Select signature genes for Connectivity Map

In [ ]:
pickle.dump(setup, file=open(setup.case_id+'_DISCoVER.p','wb'))

In [7]:
setup = pickle.load(file=open(setup.case_id+'_DISCoVER.p','rb'))

In [6]:
from drug_suggestion.expression.cmap import make_cmap_genesets, write_cmap_genesets
from drug_suggestion.expression.cmap import read_cmap_gct, load_cmap_drug_to_cids
from drug_suggestion.drug_annotation import subset_to_reasonable_drugs
from drug_suggestion.expression.discover import discover_from_expression, plot_discover_from_expression
from drug_suggestion.expression.controls import load_control_exp

@genepattern.build_ui(
  description="This function parses CMap's results.",
  parameters={
    "setup": {"default": "setup",
              "hide": False,
              "description": "The variable which has the setup information"},
    "output_var": {
        "default": "setup",
        "hide": True,
    },
})
def make_cmap_slide(setup):
    log('About to parse CMap results.')
    patient_exp = pd.read_csv(setup.patient_gexp_file, index_col=0)
    control_exp = load_control_exp(setup.expression_control)
    cmap_genesets = make_cmap_genesets(patient_exp, control_exp)
    write_cmap_genesets(cmap_genesets, setup.cmap_out_dir)

    # must match path to downloaded .gct file
    cmap_gct = os.path.join(setup.cmap_out_dir, 'cmap_result.gct')

    if os.path.exists(cmap_gct):
        cmap_ranked_drugs = read_cmap_gct(cmap_gct)
        cmap_ranked_drugs.columns = [setup.case_id]
        cmap_ranked_drugs.to_csv(setup.cmap_all_ranked_drugs_file)
        cmap2cid = load_cmap_drug_to_cids()
        cmap_reasonable = subset_to_reasonable_drugs(cmap_ranked_drugs.T, 
                                   cmap2cid,
                                   out_prefix='cmap.{}'.format(setup.expression_control), 
                                   out_dir=setup.cmap_out_dir).sort_values(by=setup.case_id, ascending=False)
        rdrugs_cmap = pd.read_csv(setup.cmap_reasonable_ranked_drugs_file, index_col=None)
        make_exp_drug_ranking_results_slide(setup.mtb_ppt_file, rdrugs_cmap, setup.expression_control, method='CMap')
        setup.rdrugs_cmap = rdrugs_cmap
        pickle.dump(setup, file=open('patients/'+setup.patient_dir+'/drug_suggestions/'+setup.case_id+'_drug_recommendations.p','wb'))
        log("done!")
    else:
        log(f"cmap_result.gct not found! (It should be present in the directiory {setup.cmap_out_dir}).")
        log("Try again if you'd like to see CMap results.")
        log("Hint, you may want to go here:")
        log("https://clue.io/l1000-query#individual")
    return setup

UIBuilder(description="This function parses CMap's results.", function_import='make_cmap_slide', name='make_cm…

DiSCoVER "supporting information" or "evidence" means in how many of the three drug databases that drug is significant.

In [7]:
def add_cmap_to_split_df(discover,cmap):
    df = discover.rename(index=str, columns={"score": "DiSCoVER", "moa":"MoA"},inplace=False)
    for index, row in cmap.iterrows():
        drug = row['drug'].lower()
        try:
            if str(df.loc[drug,'CMAP']) != 'nan':
                #drug already exists and a CMAP score has been added
                df.loc[drug,'CMAP'] = np.nanmean([row['score'], df.loc[drug,'CMAP']])
            else:
                # drug already exists but a CMAP score has not been added
                df.loc[drug,'CMAP'] = row['score']
        except KeyError:
            # drug didn't exist therefore a CMAP score had not been added
            df.loc[drug,'CMAP'] = row['score']    
            df.loc[drug,'MoA'] = row['moa']
        if str(df.loc[drug,'evidence'])=='nan': #numpy is not letting me use np.isnan()
            df.loc[drug,'evidence'] = '...'+sign_to_letter[str(np.sign(row['score']))]
        else:
            df.loc[drug,'evidence'] = str(df.loc[drug,'evidence'])+sign_to_letter[str(np.sign(row['score']))]
    
    #update those rows which are not in cmap
    for index, row in df.iterrows():
        if len(row['evidence'])==3:
            df.loc[index,'evidence']  = df.loc[index,'evidence']+'.'            
    
    return df[['drug','MoA','GDSC','CTRP','CCLE','DiSCoVER','CMAP','evidence']]

def rank_combined_df(df):
#     df['average'] = df.drop(['MoA','GDSC','CTRP','CCLE','support'],axis=1,inplace=False).mean(axis=1,skipna=True).round(3)
    df.sort_values(by=['DiSCoVER'],ascending=False,axis=0,inplace=True)
    df['DiSCoVER rank'] = range(1, len(df) + 1)
    df.sort_values(by=['CMAP'],ascending=False,axis=0,inplace=True)
    df['CMAP rank'] = range(1, len(df) + 1)
    df['Average rank'] = df[['DiSCoVER rank','CMAP rank']].mean(axis=1)
    df.sort_values(by=['Average rank'],ascending=True,axis=0,inplace=True)
    df = df[((df['DiSCoVER']>0.001).values) & ((df['CMAP']>0.001).values)]
#     combined_df = combined_df[((combined_df['DiSCoVER']>0.001).values) & ((combined_df['CMAP']>0.001).values)]

    return df.drop(['GDSC','CTRP','CCLE'],axis=1,inplace=False)

@genepattern.build_ui(
  description="This function merges the results of DiSCoVER and CMap.",
  parameters={
    "setup": {"default": "setup",
              "hide": False,
              "description": "The variable which has the setup information"},
    "output_var": {
        "default": "setup",
        "hide": True,
    },
})
def merge_discover_and_cmap(setup):
    log('Merging results from DiSCoVER and CMap.')
    combined_df = add_cmap_to_split_df(discover=setup.df,cmap=setup.rdrugs_cmap)
    to_slide = rank_combined_df(combined_df)
    make_intersection_slide(setup.mtb_ppt_file, to_slide, setup.expression_control, method='DiSCoVER ∩ CMap')
    
    log("Done Merging results from DiSCoVER and CMap!")
    log("Saving combined_df and to_slide on setup variable")
    setup.combined_df = combined_df
    setup.to_slide = to_slide
    pickle.dump(setup, file=open('patients/'+setup.patient_dir+'/drug_suggestions/'+setup.case_id+'_drug_recommendations_merged.p','wb'))
    log("Done done!")
    return setup

UIBuilder(description='This function merges the results of DiSCoVER and CMap.', function_import='merge_discove…

In [ ]:
setup.to_slide

## Upload gene expression data

In [ ]:
# exp_upload_command = 'dx upload --path /"{}"/"{}" "{}"'.format(patient_dir, os.path.basename(patient_gexp_file), patient_gexp_file)
# subprocess.check_call(exp_upload_command, shell=True)

In [9]:
# 2018-12-12

In [12]:
setup = pickle.load(file=open('patients/'+setup.patient_dir+'/drug_suggestions/'+setup.case_id+'_drug_recommendations_merged.p','rb'))

In [14]:
setup.combined_df

,drug,MoA,GDSC,CTRP,CCLE,DiSCoVER,CMAP,evidence,DiSCoVER rank,CMAP rank,Average rank
navitoclax,ctrp_navitoclax,"Bcl-2 family inhibitor: esp Bcl-xL, Bcl-2 and ...",0.469,0.262,NaN,0.366,0.471000,++.+,11,3,7.0
darinaparsin,ctrp_darinaparsin,organic arsenic compound composed of dimethyla...,NaN,0.328,NaN,0.328,0.207000,.+.+,17,5,11.0
tandutinib,ctrp_tandutinib,"FLT3 antagonist, also inhibits PDGFR and c-Kit,",NaN,0.247,NaN,0.247,0.605000,.+.+,51,2,26.5
cimetidine,ctrp_cimetidine,(Tagamet) Antacid and antihistamine,NaN,0.254,NaN,0.254,0.034500,.+.+.,49,13,31.0
rucaparib,gdsc_Rucaparib,"PARP inhibitor, inhibits DNA repair",0.371,NaN,NaN,0.371,-0.000000,+...,10,66,38.0
phenformin,gdsc_Phenformin,Anti-diabetic AMPK Activator - mTOR Inhibitor ...,0.209,NaN,NaN,0.209,0.098000,+..+,68,8,38.0
sitagliptin,ctrp_sitagliptin,"used for Treating type 2 diabetes, works by in...",NaN,0.317,NaN,0.317,-0.000000,.+..,23,61,42.0
lenalidomide,gdsc_Lenalidomide,"derivative of thalidomide, which interacts wit...",0.289,NaN,NaN,0.289,-0.000000,+...,34,53,43.5
cyclophosphamide,ctrp_cyclophosphamide,"alkylating agent, nitrogen mustard family, wor...",NaN,0.324,NaN,0.324,-0.001000,.+.-,20,69,44.5
olaparib,ctrp_olaparib,PARP Inhibitor,0.353,0.298,NaN,0.326,-0.005000,++.-,18,78,48.0
